<a href="https://colab.research.google.com/github/lephuocdat2000/DeepLearning-and-Application/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from gensim.models import KeyedVectors
import numpy as np

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Load model

Tải pretrained model cho Tiếng Việt

In [5]:
!pip install fasttext

     |████████████████████████████████| 71kB 2.7MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3097057 sha256=72fc704dfb2237aff0c893e158b09f0c591c01658389a8d6ddce1d8744be60bb
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [6]:
import fasttext.util
fasttext.util.download_model('vi',if_exists='ignore')


Load

In [ ]:
ft = fasttext.load_model('cc.vi.300.bin')

#Load data

In [15]:
%cd /content/drive/MyDrive/Advanced-CV/
!rm -rf person 

/content/drive/MyDrive/Advanced-CV


Read data from file.txt